## Depedencies

In [15]:
from get_processed_data import *
import os
import requests
import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

2025-02-27 05:09:17.732648: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-27 05:09:17.763343: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-27 05:09:17.953654: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-27 05:09:18.210312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-27 05:09:18.424250: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

## Get Data

In [70]:
# Obtener los datos planteados para el ejercicio. Si se desea el detalle de las
# funciones, ver el archivo "get_processed_data"

# Nota: El codigo se ejecuto para guardar el archivo en una ruta definida, no obstante,
# podra ejecutarlo para validar la funcionalidad

df = get_data()
num_f = numerical_feature_selection(df,quantitative_variables,target_value)
cat_f = categorical_feature_selection(df,categorical_variables,target_value)
df_processed = pd.concat([num_f , cat_f],axis=1)
df_processed.columns = df_processed.columns.str.lower()
df_processed.columns = df_processed.columns.str.lower().str.strip().str.replace(" ", "_")
df_processed.to_csv("./data/processed_data/df_processed.csv",index=False)

Dataframe was loaded
---------------------------
Selected Numerical Values
                              Columns  Retain
0                           Elevation    True
1                              Aspect   False
2                               Slope    True
3    Horizontal_Distance_To_Hydrology   False
4      Vertical_Distance_To_Hydrology   False
5     Horizontal_Distance_To_Roadways    True
6                       Hillshade_9am   False
7                      Hillshade_Noon   False
8                       Hillshade_3pm   False
9  Horizontal_Distance_To_Fire_Points    True
---------------------------
Selected Categorical Values
                      Columns  Retain
0       Wilderness_Area_Cache    True
1   Wilderness_Area_Commanche   False
2       Wilderness_Area_Neota   False
3       Wilderness_Area_Rawah   False
4             Soil_Type_C2702   False
5             Soil_Type_C2703    True
6             Soil_Type_C2704    True
7             Soil_Type_C2705    True
8             Soil_Ty

/app/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Pipeline

### Parameters

In [100]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'
# directory of the raw data files
_data_root = './data/processed_data'
# path to the raw training data
_data_filepath = os.path.join(_data_root, 'processed_data.csv')

## Download data
os.makedirs(_data_root, exist_ok=True)

context = InteractiveContext(pipeline_root=pipeline_root)

### Generate Examples

In [101]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = CsvExampleGen(input_base=data_root)

In [102]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 24
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Statistics

In [103]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 25
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [104]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])